### <span style="font-weight:bold; font-family:monospace; font-size:18px;">Step 1 - Topic: Forecast the future demand of short courses and expert courses in 2024. </span>


### <span style="font-weight:bold; font-family:monospace; font-size:18px;">Step 2 - Data Collection </b>

###     <pre style="font-weight:bold; font-size:17px">Sources:</pre>
<ul style="line-height: 1.5; font-family:monospace">
<li>Short course schools</li>
  <ul>
    <li>CSTAD</li>
    <li>Ant Training center</li>
    </ul>
<li>Survey</li>
<li>Job announcement website</li>
</ul>


<span style="font-weight: bold; color: blue; font-family:monospace">1.   Scrape data from Ant Training center</span>

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.error
from urllib.request import urlopen

# Define the URL of the main page with the list of students
main_page_url = "http://training.antkh.com/students/"

# Send an HTTP GET request to the main page and parse it with Beautiful Soup
main_page_response = urlopen(main_page_url)
main_page_soup = BeautifulSoup(main_page_response, "lxml")

# Initialize a list to store student data
student_data_list = []

# Find all the student rows in the main page
student_rows = main_page_soup.find_all("tr")

list_rows = []

# Loop through each student row
for row in student_rows[1:]:
    cells = row.find_all('td')
    str_cells = str(cells)

    # Clean the HTML tags from the cell content
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '', str_cells))

    # Initialize variables for Khmer name and University
    km_name = ""
    university = ""

    # Split the cleaned cell content into Khmer name and University
    parts = clean2.split(',')
    
    if len(parts) >= 2:
        km_name, university = parts[0], parts[1]

    # Extract the href attribute from the first "a" element within the row
    student_link = cells[0].find("a")
    href_value = student_link.get("href")
    try:
   
         # Construct the URL for the detailed student page
        detailed_page_url = f"http://training.antkh.com/students/{href_value}"

        # Send an HTTP GET request to the detailed page and parse it with Beautiful Soup
        detailed_page_response = urlopen(detailed_page_url)
        detailed_page_soup = BeautifulSoup(detailed_page_response, "lxml")

        # Extract additional information from the detailed page
        academic_info_elem = detailed_page_soup.find("div", id="c_pAcademic")
        academic_info = academic_info_elem.get_text(strip=True) if academic_info_elem else ""
        academic_year_match = re.search(r'(\d{4})', academic_info)
        academic_year = academic_year_match.group(1) if academic_year_match else 0

        study_info_elem = detailed_page_soup.find("div", class_="study-info")
        study_info_items = study_info_elem.find_all("li")
        courses = [item.get_text() for item in study_info_items]
        courses_str = ', '.join(courses)

        profile_content = detailed_page_soup.find("div", class_="inner-content")
        eng_name = profile_content.find("h2").get_text()
    except urllib.error.URLError as e:
        print(f"Failed to fetch the page: {e}")

    # Append the extracted data to the list
    list_rows.append([eng_name, km_name, university, courses_str, academic_year])

# Create a DataFrame with the extracted data
df = pd.DataFrame(list_rows, columns=["English Name", "Khmer Name", "University", "Courses", "Academic Year", ])

# Remove square brackets from Khmer Name, University, and Courses columns
df["Khmer Name"] = df["Khmer Name"].str.strip("[")
df["University"] = df["University"].str.strip("]")
df["Courses"] = df["Courses"].str.strip("[]")

# New DataFrame containing only rows with academic year > 2020
filtered_df = df[df["Academic Year"].astype(int) > 2020]

# Filter 100 records from each of the academic years 2017, 2018, and 2019
academic_years = [2017, 2018, 2019]
filtered_records = []

for year in academic_years:
    records_for_year = df[pd.to_numeric(df["Academic Year"], errors='coerce', downcast='integer') == year]
    if len(records_for_year) >= 100:
        random_records = records_for_year.sample(n=100, random_state=42)
        filtered_records.append(random_records)

# Concatenate the filtered records to the existing filtered_df
filtered_df = pd.concat([filtered_df] + filtered_records)

# Save the filtered data to a CSV file
filtered_df.to_csv("ant_training_students.csv", index=False)
filtered_df.head(100)

<span style="font-weight: bold; color: blue; font-family:monospace">2.   Import data from CSTAD short course, survey data and Ant training data</span>

In [85]:
import pandas as pd

cstad_file_path = 'CSTAD_Short_Course_Data.csv'
cstad_df = pd.read_csv(cstad_file_path)

ant_file_path = 'ant_training_students.csv'
ant_df = pd.read_csv(ant_file_path)

survey_file_path = 'IT_Course_Survey.csv'
survey_df = pd.read_csv(survey_file_path)

<span style="font-weight: bold; color: blue; font-family:monospace">3.   More data cleansing on Ant training data</span>

In [86]:
# Split the 'Courses' column by ',' and create a new DataFrame
split_df = ant_df['Courses'].str.split(',').explode().reset_index(drop=True)

# Duplicate the other columns for the split rows
new_df = ant_df.drop(columns=['Courses']).loc[ant_df.index.repeat(ant_df['Courses'].str.count(',') + 1)].reset_index(drop=True)

# Assign the split values to the 'Courses' column
new_df['Courses'] = split_df

# Remove the 'Khmer Name' column
new_df.drop(columns="Khmer Name", inplace=True)

# Add a new column "Short course school" with the value "ANT"
new_df['Short course school'] = 'ANT'

# Handle potential missing or invalid values
new_df['Courses'].fillna('', inplace=True)

# Modify course
new_df['Courses'] = new_df['Courses'].str.strip()
new_df.rename(columns={'Courses': 'Course'}, inplace=True)
new_df = new_df[new_df['Course'].str.strip() != '']

course_mapping = {
    'HTML+HTML5&CSS': 'Web Design',
    'Website Design With CSS': 'Web Design',
    'CSS3 & Bootstrap': 'Web Design',
    'C++OOP': 'C++',
    'JavaScript + Jquery': 'Web Design',
    'C# Beginning': 'C#',
    'Node.js +Express': 'Node JS',
    'Java Programing' : 'Java',
    'Mobile App (Flutter)': 'Flutter',
    'C/C++': 'C++',
    'Web Development': 'Full stack',
    'C# Database Programming': 'C#',
    'OOP PHP': 'PHP',
    'Android Application Development': 'Flutter',
    'HTML': 'Web Design',
    'PHP & My SQL': 'PHP'
    }

new_df['Course'] = new_df['Course'].replace(course_mapping)
new_df['Course'] = new_df['Course'].str.strip()
# Remove duplicate row and Change student name column
new_df = new_df.drop_duplicates()
new_df.rename(columns={'English Name': 'Fullname'}, inplace=True)

# Save the final DataFrame to a new CSV file
new_df.to_csv('ant_training_students_v2.csv', index=False)
new_df.to_excel('ant_training_students_v2.xlsx', index=False)

/Users/jessicasun/anaconda3/lib/python3.11/site-packages/pandas/core/series.py:893: RuntimeWarning: invalid value encountered in cast
  return np.asarray(self._values, dtype)


<span style="font-weight: bold; color: blue; font-family:monospace">4.   Combine data from 3 sources</span>

In [82]:
new_df.rename(columns={'Course': 'Course', 'Academic Year': 'Academic Year'}, inplace=True)
survey_df.rename(columns={'Course': 'Course', 'Academic Year': 'Academic Year'}, inplace=True)

# Concatenate DataFrames
final_df = pd.concat([cstad_df, new_df, survey_df], ignore_index=True)
final_df.to_csv('final_dataset.csv', index=False)
final_df.to_excel('final_dataset.xlsx', index=False)